In [ ]:

%matplotlib inline
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import plotly.express as px

import seaborn as sns
import matplotlib.pyplot as plt



import pandas as pd
import numpy as np



In [ ]:

def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(history.epoch, np.array(history.history['loss']), 
           label='Train')
    plt.plot(history.epoch, np.array(history.history['val_loss']),
           label = 'Val')
    plt.legend()
    plt.ylim([0,max(history.history['val_loss'])])
    plt.savefig("History ANN.eps", format = 'eps')

In [ ]:
def plot_history2(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(history.epoch, np.array(history.history['accuracy']), 
           label='Train')
    plt.plot(history.epoch, np.array(history.history['val_accuracy']),
           label = 'Val')
    plt.legend()
    plt.ylim([0,max(history.history['val_accuracy'])])
    plt.savefig("history_cnn.png")

In [ ]:
df1 = pd.read_csv("Super_new_99.csv")

In [ ]:
df1.head()

In [ ]:
profile = ProfileReport(df1, title = "Report", explorative = "true")

In [ ]:
profile.to_file("output.html")

In [ ]:
del df1['url']

In [ ]:
del df1['label']

In [ ]:
df1

In [ ]:
df2 = df1.iloc[: , 0:1]

In [ ]:
df2

In [ ]:
del df1['result']

In [ ]:
df1

In [ ]:
y = df2.iloc[: , 0:1].values

In [ ]:
print(y)

In [ ]:
df1.shape

In [ ]:
[df1.dtypes]

# Features Selection

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_trash = VarianceThreshold(threshold = (0.02))
var_trash.fit(df1)

var_trash.get_support()

#No Zero Variance

### Correlation


In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))
cor = df1.corr()
sns.heatmap(cor,annot=True,cmap = plt.cm.CMRmap_r)
plt.show()
plt.savefig("correlation.eps", format = 'eps')

In [ ]:
def correlation(dataset, threshold):
  col_corr = set()
  corr_matrix=dataset.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if abs(corr_matrix.iloc[i,j]) > threshold:
        colname = corr_matrix.columns[i]
        col_corr.add(colname)
  
  return col_corr

In [ ]:
corr_feature = correlation(df1,0.85)

corr_feature

In [ ]:
del df1["count-letters"]


In [ ]:
df1

In [ ]:
plt.figure(figsize=(12,10))
cor = df1.corr()
sns.heatmap(cor,annot=True,cmap = plt.cm.CMRmap_r)
plt.show()
plt.savefig("correlation_after.eps", format = 'eps')

In [ ]:
df1.to_csv('selected_features.csv', index=False) 

In [ ]:
x = df1.iloc[: , :].values

In [ ]:
x.shape

In [ ]:
df1.shape

In [ ]:
x = x.astype(np.float32)

# Train - Test Split 

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

# Feature Scaling 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
print(x)

In [ ]:
print(x_train)

# Over Sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
over_sampler = RandomOverSampler(0.75)
x_train, y_train = over_sampler.fit_resample(x_train, y_train)

# Trees 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_random_forest = RandomForestClassifier(random_state = 0)
classifier_random_forest.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred_random_forest = classifier_random_forest.predict(x_test)


cm_random_forest = confusion_matrix(y_test , y_pred_random_forest)
print(cm_random_forest)
accuracy_score(y_test, y_pred_random_forest)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier_d_tree = DecisionTreeClassifier( criterion = 'entropy', random_state = 0, max_depth = None)
classifier_d_tree.fit(x_train, y_train)


y_pred_d_tree = classifier_d_tree.predict(x_test)


cm_d_tree = confusion_matrix(y_test , y_pred_d_tree)
print(cm_d_tree)
accuracy_score(y_test, y_pred_d_tree)

# Perceptron (Keras Tuner)

In [ ]:
import tensorflow as tf

In [ ]:
from keras_tuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=18,
                                            max_value=4096,
                                            step=32),
                               activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='',
    project_name='Spam URL1')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train, y_train,
             epochs=100,
             validation_data=(x_test, y_test))

In [ ]:
tuner.results_summary()

In [ ]:
ann = tf.keras.models.Sequential()




ann.add(tf.keras.layers.Dense(units=512, activation='relu', kernel_initializer = 'he_uniform'))


ann.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_initializer = 'he_uniform'))


ann.add(tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer = 'he_uniform' ))


ann.add(tf.keras.layers.Dense(units=32, activation='relu', kernel_initializer = 'he_uniform'  ))


ann.add(tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer = 'he_uniform'  ))

ann.add(tf.keras.layers.Dense(units=2, activation='relu', kernel_initializer = 'he_uniform'  ))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
ann.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history_ann = ann.fit(x_train, y_train, epochs = 50, batch_size = 64 , validation_split=0.2, verbose=1)

In [ ]:
plot_history(history_ann)

In [ ]:
plot_history2(history_ann)

In [ ]:
y_pred_ann = ann.predict(x_test)
y_pred_ann = (y_pred_ann > 0.5)
print(y_pred_ann)

cm_ann = confusion_matrix(y_test, y_pred_ann)
print(cm_ann)
accuracy_score(y_test,  y_pred_ann)

# Conv1D

### Reshape dataset for CNN

In [ ]:
x_train_reshaped = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test_reshaped = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
n_timesteps = x_train_reshaped.shape[1]
n_features  = x_train_reshaped.shape[2]

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Input(shape=(n_timesteps,n_features)))
cnn.add(tf.keras.layers.Conv1D( 128, kernel_size=2, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv1D( 64, kernel_size=2, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv1D( 64, kernel_size=2, activation='relu'))
cnn.add(tf.keras.layers.MaxPool1D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=3471, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.5))

cnn.add(tf.keras.layers.Dense(units=847, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.5))

cnn.add(tf.keras.layers.Dense(units=847, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.5))

cnn.add(tf.keras.layers.Dense(units=521, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(.5))

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer = 'adamax', loss = 'binary_crossentropy', metrics = ['accuracy'] )

In [ ]:
history = cnn.fit(x_train_reshaped, y_train, epochs = 10, batch_size = 64 , validation_split=0.2, verbose=1)

In [ ]:
plot_history(history)

In [ ]:
y_pred_cnn = cnn.predict(x_test)
y_pred_cnn = (y_pred_cnn > 0.5)
print(y_pred_cnn)

cm_Conv1D = confusion_matrix(y_test, y_pred_cnn)
print(cm_Conv1D)
accuracy_score(y_test,  y_pred_cnn)

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred_cnn))

# Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_logistic = LogisticRegression(random_state = 10)
classifier_logistic.fit(x_train, y_train)


from sklearn.model_selection import GridSearchCV

y_predict = classifier_logistic.predict(x_test)


from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test , y_predict)
print(cm)
accuracy_score(y_test, y_predict)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier(n_neighbors = 9, metric = 'minkowski', p = 2)
classifier_knn.fit(x_train, y_train)


y_pred_knn = classifier_knn.predict(x_test)


cm_knn = confusion_matrix(y_test , y_pred_knn)
print(cm_knn)
accuracy_score(y_test, y_pred_knn)

# Gaussian NB

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier_NB = GaussianNB()
classifier_NB.fit(x_train, y_train)

y_pred_NB = classifier_NB.predict(x_test)

cm_NB = confusion_matrix(y_test, y_pred_NB)
print(cm_NB)
accuracy_score(y_test, y_pred_NB)

# XGBoost

In [ ]:
from xgboost import XGBClassifier
classifier_XGB = XGBClassifier()
classifier_XGB.fit(x_train, y_train)

y_pred_XGB = classifier_XGB.predict(x_test)
cm_XGB = confusion_matrix(y_test, y_pred_XGB)
print(cm_XGB)
accuracy_score(y_test, y_pred_XGB)